Import all the necessary stuff,

In [4]:
import matplotlib.pyplot as plt
from kadlu.sound.sound_propagation import TLCalculator, Seafloor
from kadlu.geospatial.ocean import Ocean

Create a simplified, uniform, seabed with the following properties: 
  
  * thickness of 2 km
  * sound speed of 1,700 m/s
  * density of 1.5 g/cm^3
  * attenuation of 0.5 dB per wavelength,

In [5]:
seaflo = Seafloor(thickness=2000, c=1700, density=1.5, loss=0.5)

Create a simplified, uniform, ocean volume which is 5 km deep everywhere

In [ ]:
o = Ocean(bathy=-10000)

In [ ]:
tl = TLCalculator(ocean=o, seafloor=s, sound_speed=1500,\
    radial_bin=1000, radial_range=10e3, angular_bin=45, vertical_bin=1000,\
    verbose=True, progress_bar=False)

In [ ]:
tl.run(frequency=10, source_depth=-9900, source_lat=45, source_lon=60)

In [ ]:
fig = tl.plot()